### Initial Setup

In [ ]:
!pip install cython
!pip install opencv-python pillow pycocotools matplotlib 

### Import library

In [2]:
#-*- coding:utf-8 -*-
import os
from os.path import exists, join, basename, splitext
import random
import time
import numpy as np

import PIL
from PIL import Image

import matplotlib
import matplotlib.pylab as plt
import matplotlib.pyplot as plt
from matplotlib.path import Path
import matplotlib.patches as patches
plt.rcParams["axes.grid"] = False

import torch
import torchvision
from torchvision import models
import torchvision.transforms as T
torch.set_grad_enabled(False)

import cv2
print(os.getcwd())

# Download Model
model = models.detection.keypointrcnn_resnet50_fpn(pretrained=True).eval()

C:\Users\min11\Desktop\coding\torch_bgrm_pose


### Step 1. Transform video to image(per seconds)

In [ ]:
# Video to Image
path = "./img/1_input_video/"
file_list = os.listdir(path)
print("{}".format(file_list))

def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()

    if hasFrames:
        h = 256
        w = 256
        resized_image = cv2.resize(image, (h, w)).astype(np.float32)
        if count < 10:
            cv2.imwrite("./img/2_input/"+'0000'+str(count)+".jpg", resized_image)     # save frame as JPG file
        elif count < 100:
            cv2.imwrite("./img/2_input/"+'000'+str(count)+".jpg", resized_image)     # save frame as JPG file
        elif count < 1000:
            cv2.imwrite("./img/2_input/"+'00'+str(count)+".jpg", resized_image)     # save frame as JPG file
        elif count < 10000:
            cv2.imwrite("./img/2_input/"+'0'+str(count)+".jpg", resized_image)     # save frame as JPG file

    return hasFrames

for file in file_list:
    vidcap = cv2.VideoCapture(path+file)
    sec = 0
    frameRate = 1 #//it will capture image in each 1 second
    count=1
    success = getFrame(sec)

    while success:
        count = count + 1
        sec = sec + frameRate
        sec = round(sec, 2)
        success = getFrame(sec)


### Step 2. Background Subtraction(YOLACT++)

In [3]:
%cd yolact
!python eval.py --trained_model=weights/yolact_resnet50_54_800000.pth --score_threshold=0.15 --top_k=1 --images="../img/2_input":"../img/3_output_bg"
%cd ..

C:\Users\min11\Desktop\coding\torch_bgrm_pose\yolact
Config not specified. Parsed yolact_resnet50_config from the file name.

Loading model... Done.

..\img\2_input\10.jpg -> ../img/3_output_bg\10.jpg
..\img\2_input\11.jpg -> ../img/3_output_bg\11.jpg
..\img\2_input\12.jpg -> ../img/3_output_bg\12.jpg
..\img\2_input\13.jpg -> ../img/3_output_bg\13.jpg
..\img\2_input\19.jpg -> ../img/3_output_bg\19.jpg
..\img\2_input\5.jpg -> ../img/3_output_bg\5.jpg
..\img\2_input\6.jpg -> ../img/3_output_bg\6.jpg
..\img\2_input\7.jpg -> ../img/3_output_bg\7.jpg
..\img\2_input\8.jpg -> ../img/3_output_bg\8.jpg
..\img\2_input\9.jpg -> ../img/3_output_bg\9.jpg
Done.
C:\Users\min11\Desktop\coding\torch_bgrm_pose


C:\Users\min11\Anaconda3\lib\site-packages\torch\jit\_recursive.py:181: UserWarning: 'lat_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn("'{}' was found in ScriptModule constants, "
C:\Users\min11\Anaconda3\lib\site-packages\torch\jit\_recursive.py:181: UserWarning: 'pred_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn("'{}' was found in ScriptModule constants, "
C:\Users\min11\Anaconda3\lib\site-packages\torch\jit\_recursive.py:181: UserWarning: 'downsample_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn("'{}' was found in ScriptModule constants, "


### Step 3. Skeleton Detection(Mask R-CNN Keypoint)

In [3]:
path = './img/3_output_bg/'
file_list = os.listdir(path)
print("{}".format(file_list))

['10.jpg', '11.jpg', '12.jpg', '13.jpg', '19.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg']


In [ ]:
THRESHOLD = 0.95

path = "./img/3_output_bg/"
file_list = os.listdir(path)
print("{}".format(file_list))

for file in file_list:

    image = Image.open("./img/3_output_bg/"+file)
    
    t = time.time()
    # Image to Tensor
    trf = T.Compose([
        T.Resize(256),
        T.ToTensor() 
        ])
    input_img = trf(image) 
    out = model([input_img])[0]

    codes = [
        Path.MOVETO,
        Path.LINETO,
        Path.LINETO
    ]

    fig, ax = plt.subplots(1, figsize=(8, 8))
    ax.imshow(image)

    for box, score, keypoints in zip(out['boxes'], out['scores'], out['keypoints']):
        # 스코어가 95% 이상인 것만 사용
        score = score.detach().numpy() # score(텐서)를 떼어내서 numpy배열로 바꾼다.

        if score < THRESHOLD:
            continue

        box = box.detach().numpy()
        keypoints = keypoints.detach().numpy()[:, :2] # numpy array로 변환하고 2개의 값(x, y)만 저장

        rect = patches.Rectangle((box[0], box[1]), box[2]-box[0], box[3]-box[1], linewidth=2, edgecolor='b', facecolor='none')
        ax.add_patch(rect) # matplotlib에 patches를 사용하여 박스를 그려준다.

        # 17 keypoints(표시할 점의 갯수를 조절가능)
        for k in keypoints[5:13]:

            circle = patches.Circle((k[0], k[1]), radius=2, facecolor='r')
            ax.add_patch(circle)
            
            ax.text(keypoints[5][0],keypoints[5][1], '5', color='green')
            ax.text(keypoints[6][0],keypoints[6][1], '6', color='yellow')
            ax.text(keypoints[7][0],keypoints[7][1], '7', color='green')
            ax.text(keypoints[8][0],keypoints[8][1], '8', color='yellow')
            ax.text(keypoints[9][0],keypoints[9][1], '9', color='green')
            ax.text(keypoints[10][0],keypoints[10][1], '10', color='yellow')
            ax.text(keypoints[11][0],keypoints[11][1], '11', color='green')
            ax.text(keypoints[12][0],keypoints[12][1], '12', color='yellow')

            head = ((keypoints[3]+keypoints[4])/2)
            ax.text(head[0],head[1], '3/4', color='yellow')

            # Draw line
            # 왼쪽어깨굴곡(6,8,12)
            verts = [keypoints[8],keypoints[6],keypoints[12]]
            path = Path(verts, codes)
            line = patches.PathPatch(path, linewidth=2, facecolor='none', edgecolor='r')
            ax.add_patch(line)


            # 왼쪽팔꿈치굴곡(5,7,9)
            verts = [keypoints[5],keypoints[7],keypoints[9]]
            path = Path(verts, codes)
            line = patches.PathPatch(path, linewidth=2, facecolor='none', edgecolor='y')
            ax.add_patch(line)

            # 오른쪽어깨굴곡(11,5,7)
            verts = [keypoints[11],keypoints[5],keypoints[7]]
            path = Path(verts, codes)
            line = patches.PathPatch(path, linewidth=2, facecolor='none', edgecolor='y')
            ax.add_patch(line)

            # 오른쪽팔꿈치굴곡(6,8,10)
            verts = [keypoints[6],keypoints[8],keypoints[10]]
            path = Path(verts, codes)
            line = patches.PathPatch(path, linewidth=2, facecolor='none', edgecolor='r')
            ax.add_patch(line)

            # 목 굴곡
            verts = [((keypoints[3]+keypoints[4])/2),((keypoints[5]+keypoints[6])/2),((keypoints[11]+keypoints[12])/2)]
            path = Path(verts, codes)
            line = patches.PathPatch(path, linewidth=2, facecolor='none', edgecolor='g')
            ax.add_patch(line)

            # 허리 굴곡
            verts = [((keypoints[5]+keypoints[6])/2), ((keypoints[11]+keypoints[12])/2), (((keypoints[11]+keypoints[12])/2)[0],((keypoints[5]+keypoints[6])/2)[1])]
            path = Path(verts, codes)
            line = patches.PathPatch(path, linewidth=1, facecolor='none', edgecolor='b')
            ax.add_patch(line)
            ax.set_axis_off()
            fig.savefig('./img/4_output_kp/'+file, bbox_inches='tight')
            
    # 각도 계산
    import math
    def getAngle(a, b, c):
        ang = math.degrees(math.atan2(c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1], a[0]-b[0]))
        return abs(ang - 360) if ang > 180 else abs(ang) #+ 360 if ang < 0 else ang

    left_sho_angle = (getAngle((keypoints[7]), (keypoints[5]), (keypoints[11])))
    right_sho_angle = (getAngle((keypoints[8]), (keypoints[6]), (keypoints[12])))
    
    left_albow_angle = (getAngle((keypoints[5]), (keypoints[7]), (keypoints[9])))
    right_albow_angle = (getAngle((keypoints[6]), (keypoints[8]), (keypoints[10])))

    
    neck_angle = getAngle(((keypoints[3]+keypoints[4])/2),((keypoints[5]+keypoints[6])/2),((keypoints[11]+keypoints[12])/2))
    if neck_angle > 90 :
        neck_angle = (180 - getAngle(((keypoints[3]+keypoints[4])/2), ((keypoints[5]+keypoints[6])/2), ((keypoints[11]+keypoints[12])/2)))/2
    elif neck_angle >= 45:
        neck_angle = neck_angle/15
    elif neck_angle < 45:
        neck_angle = neck_angle

    trunk_angle = getAngle(((keypoints[5]+keypoints[6])/2), ((keypoints[11]+keypoints[12])/2),(((keypoints[11]+keypoints[12])/2)[0],((keypoints[5]+keypoints[6])/2)[1]))/2
    print("작업명 : "+file)
    print("왼쪽 어깨 굴곡 각도       : "+str(round(left_sho_angle, 2))+" 도")
    print("왼쪽 팔꿈치 굴곡 각도     : "+str(round(left_albow_angle, 2))+" 도")
    print("오른쪽 어깨 굴곡 각도     : "+str(round(right_sho_angle, 2))+" 도")
    print("오른쪽 팔꿈치 굴곡 각도   : "+str(round(right_albow_angle, 2))+" 도")
    print("목 굴곡 각도              : "+str(round(neck_angle, 2))+" 도")
    print("허리 굴곡 각도            : "+str(round(trunk_angle, 2))+" 도")


    print('executed in %.3fs' % (time.time() - t))

['10.jpg', '11.jpg', '12.jpg', '13.jpg', '19.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg']
